<h1>Block Group Population Centers</h1>

In [2]:
import pandas as pd
import numpy as np
import pickle
from shapely.geometry import Point
import geopandas as gpd
import addfips
import pickle
af=addfips.AddFIPS()

In [2]:
# Read in census tract population and coordinate data
pop_centers = pd.read_csv('../../Obstetric accessibility visualization/Data/BG_pop_center.csv')
# pop_centers = pd.read_csv('../../example1.csv')

In [3]:
pop_centers.head() 

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE
0,19,1,960100,1,1374,41.458179,-94.342516
1,19,1,960100,2,613,41.438395,-94.575000
2,19,1,960100,3,789,41.500753,-94.646195
3,19,1,960200,1,550,41.261736,-94.338260
4,19,1,960200,2,1061,41.221432,-94.524858


In [4]:
# Georgia shapefile
iowa_BG = gpd.read_file('../../Obstetric accessibility visualization/Data/tl_2017_13_bg.shp')

In [5]:
pop_centers['COUNTYFP'] = pop_centers.apply(lambda row: str(int(row['COUNTYFP'])).zfill(3),axis=1)
pop_centers['FIPS'] = pop_centers.apply(lambda row: str(row['STATEFP']) + str(row['COUNTYFP']),axis=1)
pop_centers['Coordinates'] = pop_centers.apply(lambda row: (row['LONGITUDE'],row['LATITUDE']),axis=1)
pop_centers = pop_centers.set_index('FIPS')

In [6]:
pop_centers.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE,Coordinates
FIPS,,,,,,,,
19001,19,001,960100,1,1374,41.458179,-94.342516,"(-94.342516, 41.458179)"
19001,19,001,960100,2,613,41.438395,-94.575000,"(-94.575, 41.438395)"
19001,19,001,960100,3,789,41.500753,-94.646195,"(-94.646195, 41.500753)"
19001,19,001,960200,1,550,41.261736,-94.338260,"(-94.33826, 41.261736)"
19001,19,001,960200,2,1061,41.221432,-94.524858,"(-94.524858, 41.221432)"


In [7]:
geometry = [Point(xy) for xy in zip(pop_centers['Coordinates'])]
pop_centers = gpd.GeoDataFrame(pop_centers, 
                        crs = iowa_BG.crs,
                        geometry = geometry)
pop_centers.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE,Coordinates,geometry
FIPS,,,,,,,,,
19001,19,001,960100,1,1374,41.458179,-94.342516,"(-94.342516, 41.458179)",POINT (-94.34252 41.45818)
19001,19,001,960100,2,613,41.438395,-94.575000,"(-94.575, 41.438395)",POINT (-94.57500 41.43839)
19001,19,001,960100,3,789,41.500753,-94.646195,"(-94.646195, 41.500753)",POINT (-94.64620 41.50075)
19001,19,001,960200,1,550,41.261736,-94.338260,"(-94.33826, 41.261736)",POINT (-94.33826 41.26174)
19001,19,001,960200,2,1061,41.221432,-94.524858,"(-94.524858, 41.221432)",POINT (-94.52486 41.22143)


In [8]:
def geography_column(row):
    county = str(int(row['COUNTYFP'])).zfill(3)
    tract = str(int(row['TRACTCE'])).zfill(6)
    s = '1500000US19' + county + tract + str(int(row['BLKGRPCE']))
    return s
    
pop_centers['Geography'] = pop_centers.apply(lambda row: geography_column(row),axis=1)
pop_centers.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,POPULATION,LATITUDE,LONGITUDE,Coordinates,geometry,Geography
FIPS,,,,,,,,,,
19001,19,001,960100,1,1374,41.458179,-94.342516,"(-94.342516, 41.458179)",POINT (-94.34252 41.45818),1500000US190019601001
19001,19,001,960100,2,613,41.438395,-94.575000,"(-94.575, 41.438395)",POINT (-94.57500 41.43839),1500000US190019601002
19001,19,001,960100,3,789,41.500753,-94.646195,"(-94.646195, 41.500753)",POINT (-94.64620 41.50075),1500000US190019601003
19001,19,001,960200,1,550,41.261736,-94.338260,"(-94.33826, 41.261736)",POINT (-94.33826 41.26174),1500000US190019602001
19001,19,001,960200,2,1061,41.221432,-94.524858,"(-94.524858, 41.221432)",POINT (-94.52486 41.22143),1500000US190019602002


In [9]:
pop_centers.to_csv('../../Obstetric accessibility visualization/Data/Cleaned_Block_Group_Pop_Centers.csv')

<h1>Reproductive Age Population By Block Group</h1>

In [10]:
bg_age = pd.read_csv('../../Obstetric accessibility visualization/Data/ACSDT5Y2021.B01001-Data.csv')


In [11]:
# Get reproductive age females
bg_age.rename(columns=bg_age.iloc[0],inplace=True)
#bg_age.drop(bg_age.index[0], inplace=True)
bg_repro_age = bg_age[['Geographic Area Name',
                            'Estimate!!Total:!!Female:!!18 and 19 years',
                            'Estimate!!Total:!!Female:!!20 years',
                            'Estimate!!Total:!!Female:!!21 years',
                            'Estimate!!Total:!!Female:!!22 to 24 years',
                            'Estimate!!Total:!!Female:!!25 to 29 years',
                            'Estimate!!Total:!!Female:!!30 to 34 years',
                            'Estimate!!Total:!!Female:!!35 to 39 years',
                            'Estimate!!Total:!!Female:!!40 to 44 years']]
bg_repro_age_copy=bg_repro_age.copy()
bg_repro_age_copy.drop(bg_repro_age.index[0], inplace=True)
bg_repro_age_copy = bg_repro_age_copy.set_index('Geographic Area Name')
bg_repro_age.head()

,Geographic Area Name,Estimate!!Total:!!Female:!!18 and 19 years,Estimate!!Total:!!Female:!!20 years,Estimate!!Total:!!Female:!!21 years,Estimate!!Total:!!Female:!!22 to 24 years,Estimate!!Total:!!Female:!!25 to 29 years,Estimate!!Total:!!Female:!!30 to 34 years,Estimate!!Total:!!Female:!!35 to 39 years,Estimate!!Total:!!Female:!!40 to 44 years
0,Geographic Area Name,Estimate!!Total:!!Female:!!18 and 19 years,Estimate!!Total:!!Female:!!20 years,Estimate!!Total:!!Female:!!21 years,Estimate!!Total:!!Female:!!22 to 24 years,Estimate!!Total:!!Female:!!25 to 29 years,Estimate!!Total:!!Female:!!30 to 34 years,Estimate!!Total:!!Female:!!35 to 39 years,Estimate!!Total:!!Female:!!40 to 44 years
1,"Block Group 1, Census Tract 9601, Adair County...",3,9,0,27,10,55,25,54
2,"Block Group 2, Census Tract 9601, Adair County...",10,0,0,0,9,33,27,21
3,"Block Group 3, Census Tract 9601, Adair County...",6,0,0,6,30,26,19,18
4,"Block Group 1, Census Tract 9602, Adair County...",4,15,0,0,0,0,26,0


In [12]:
bg_repro_age_copy= bg_repro_age_copy.iloc[1:]
bg_repro_age_copy = bg_repro_age_copy.astype(int)
bg_repro_age_copy['Total Female, 18-44'] = bg_repro_age_copy.iloc[:,1:8].sum(axis=1)

In [13]:
for i in list(bg_repro_age_copy['Total Female, 18-44'].unique()):
    if isinstance(i,int):
        print(i)

In [14]:
# get population of reproductive age women in census tract
p = {}
total=0
for b in bg_repro_age_copy.index:
    p[b] = int(bg_repro_age_copy['Total Female, 18-44'][b])
    total+=p[b]
total

492680

In [15]:
# save dictionary
with open('../../Obstetric accessibility visualization/Data/p_BG.pkl', 'wb') as fp:
    pickle.dump(p, fp)

In [16]:
import pickle

with open('../../Obstetric accessibility visualization/Data/p_BG.pkl', 'rb') as file:
    df = pickle.load(file)


In [17]:
df

{'Block Group 2, Census Tract 9601, Adair County, Iowa': 90,
 'Block Group 3, Census Tract 9601, Adair County, Iowa': 99,
 'Block Group 1, Census Tract 9602, Adair County, Iowa': 41,
 'Block Group 2, Census Tract 9602, Adair County, Iowa': 181,
 'Block Group 1, Census Tract 9603, Adair County, Iowa': 93,
 'Block Group 2, Census Tract 9603, Adair County, Iowa': 21,
 'Block Group 3, Census Tract 9603, Adair County, Iowa': 89,
 'Block Group 4, Census Tract 9603, Adair County, Iowa': 166,
 'Block Group 1, Census Tract 9501, Adams County, Iowa': 104,
 'Block Group 2, Census Tract 9501, Adams County, Iowa': 80,
 'Block Group 1, Census Tract 9502, Adams County, Iowa': 143,
 'Block Group 2, Census Tract 9502, Adams County, Iowa': 136,
 'Block Group 1, Census Tract 9601, Allamakee County, Iowa': 27,
 'Block Group 2, Census Tract 9601, Allamakee County, Iowa': 172,
 'Block Group 1, Census Tract 9602, Allamakee County, Iowa': 91,
 'Block Group 2, Census Tract 9602, Allamakee County, Iowa': 159,
 